# Preparation of GWR - Data

## Libraries and Settings

In [1]:
# Libraries
import os
import json
import folium
import requests
import numpy as np
import pandas as pd
import geopandas as gpd
from datetime import date

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

print(os.getcwd())

/Users/ricostaedeli/Documents/ZHAW/GIS/gis_unterricht/Gis_projekt


## Download GWR data

Source: https://public.madd.bfs.admin.ch

In [12]:
canton = "ow"
# Download latest file for the Canton of Zurich
url = f"https://public.madd.bfs.admin.ch/buildings_{canton}.geojson"
response = requests.get(url)

# Ensure response is valid
if response.status_code == 200:
    # Open file in write mode and write the response content
    with open(f'DataBig/buildings_{canton}.geojson', 'wb') as file:
        file.write(response.content)
else:
    print(f"Failed to download file, status code: {response.status_code}")

# Convert geojson to csv
This is necessary to import in postgis

In [13]:
import geopandas as gpd

# Path to your GeoJSON file
geojson_path = f'DataBig/buildings_{canton}.geojson'
# Desired output CSV file path
csv_path = f'DataBig/output_{canton}.csv'

# Read the GeoJSON file
gdf = gpd.read_file(geojson_path)

gdf

,egid,buildingStatus,buildingCategory,buildingClass,municipalityNumber,municipalityName,canton,geometry
0,101235305,1007,1020,1110,1401,Alpnach,OW,POINT (2662391.000 1198346.000)
1,101235307,1004,1020,1110,1401,Alpnach,OW,POINT (2662273.495 1198664.438)
2,101235308,1004,1020,1122,1401,Alpnach,OW,POINT (2663069.775 1200010.861)
3,101235310,1004,1020,1110,1401,Alpnach,OW,POINT (2662170.316 1198226.835)
4,101235311,1004,1020,1110,1401,Alpnach,OW,POINT (2662135.736 1198271.709)
...,...,...,...,...,...,...,...,...
21924,9082198,1004,1040,0,1407,Sarnen,OW,POINT (2661576.416 1194345.086)
21925,9082200,1004,1060,0,1407,Sarnen,OW,POINT (2661535.114 1194255.211)
21926,9082201,1004,1060,0,1407,Sarnen,OW,POINT (2661735.402 1194681.900)
21927,9082202,1004,1060,0,1407,Sarnen,OW,POINT (2661570.119 1194122.619)


In [14]:
gdf['geometry_wkt'] = gdf['geometry'].apply(lambda x: x.wkt)

# Assuming your data is in a CRS like EPSG:2056 (Swiss coordinate system)
gdf.crs = "EPSG:2056"  # Set the correct CRS

# Convert to geographic coordinates (WGS84)
gdf = gdf.to_crs(epsg=4326)

# Extract longitude and latitude into separate columns
gdf['longitude'] = gdf.geometry.x
gdf['latitude'] = gdf.geometry.y


# Save the DataFrame to CSV
gdf.to_csv(csv_path, index=False)

gdf

,egid,buildingStatus,buildingCategory,buildingClass,municipalityNumber,municipalityName,canton,geometry,geometry_wkt,longitude,latitude
0,101235305,1007,1020,1110,1401,Alpnach,OW,POINT (8.25795 46.93328),POINT (2662391 1198346),8.257952,46.933279
1,101235307,1004,1020,1110,1401,Alpnach,OW,POINT (8.25645 46.93615),POINT (2662273.495 1198664.438),8.256452,46.936154
2,101235308,1004,1020,1122,1401,Alpnach,OW,POINT (8.26710 46.94819),POINT (2663069.775 1200010.861),8.267095,46.948190
3,101235310,1004,1020,1110,1401,Alpnach,OW,POINT (8.25504 46.93223),POINT (2662170.316 1198226.835),8.255038,46.932228
4,101235311,1004,1020,1110,1401,Alpnach,OW,POINT (8.25459 46.93263),POINT (2662135.736 1198271.709),8.254590,46.932635
...,...,...,...,...,...,...,...,...,...,...,...
21924,9082198,1004,1040,0,1407,Sarnen,OW,POINT (8.24671 46.89737),POINT (2661576.416 1194345.086),8.246714,46.897368
21925,9082200,1004,1060,0,1407,Sarnen,OW,POINT (8.24616 46.89656),POINT (2661535.114 1194255.211),8.246160,46.896563
21926,9082201,1004,1060,0,1407,Sarnen,OW,POINT (8.24885 46.90038),POINT (2661735.402 1194681.9),8.248846,46.900383
21927,9082202,1004,1060,0,1407,Sarnen,OW,POINT (8.24660 46.89537),POINT (2661570.119 1194122.619),8.246602,46.895367


In [8]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')

-----------------------------------
POSIX
Darwin | 23.5.0
Datetime: 2024-05-24 09:36:37
Python Version: 3.12.3
-----------------------------------
